In [ ]:
pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import random

# Step 1: Create a larger dataset with 1000+ samples

def generate_synthetic_data():
    """Function to generate synthetic Python code snippets with errors and correct code."""
    correct_code = [
        'print("Hello World")',
        'if x == 10: print("Number 10")',
        'def add(a, b): return a + b',
        'x = 5 + 3',
        'while True: print("Looping")',
        'for i in range(5): print(i)',
        'class MyClass: def method(self): pass',
        'try: pass except: pass',
        'def square(x): return x * x',
        'import math; print(math.sqrt(16))'
    ]

    error_code = [
        'if x == 10 print("Number 10")',  # Missing colon
        'def add(a, b) return a + b',     # Missing colon
        'print("Hello World"',            # Missing closing parenthesis
        'for i in range(5) print(i)',     # Missing colon
        'while True print("Endless loop")',  # Missing colon
        'x = 5 + "text"',                 # Type error
        'class MyClass: def method(self)' # Syntax error (missing body)
    ]

    # Generate 1000+ code samples by randomly selecting correct and error code samples
    dataset = []
    for _ in range(1000):
        if random.random() > 0.5:
            dataset.append({'code': random.choice(correct_code), 'label': 0})  # 0 for Correct
        else:
            dataset.append({'code': random.choice(error_code), 'label': 1})    # 1 for Syntax Error

    return dataset

# Step 2: Tokenizer for splitting the code into tokens
def tokenize_code(code):
    """Tokenizes Python code into syntax elements."""
    keywords = ['def', 'return', 'if', 'else', 'while', 'for', 'in', 'print', 'class', 'try', 'except', 'import', 'as', 'True', 'False', 'None']
    operators = ['=', '==', '!=', '>', '<', '>=', '<=', '+', '-', '*', '/', '%', '**']
    delimiters = ['(', ')', '{', '}', '[', ']', ':', ',', ';']

    # Regex pattern to match keywords, operators, and other syntax elements
    token_pattern = r'|'.join([
        r'\b(?:' + '|'.join(keywords) + r')\b',   # Match keywords
        r'\b(?:' + '|'.join(operators) + r')\b',  # Match operators
        r'[(){}[\],;]',                         # Match delimiters
        r'\b(?:[a-zA-Z_][a-zA-Z0-9_]*|\d+)\b'    # Match identifiers and numbers
    ])

    tokens = re.findall(token_pattern, code)
    return tokens

# Step 3: Prepare Dataset for Training
train_data = generate_synthetic_data()

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['code'], padding="max_length", truncation=True)

train_dataset = Dataset.from_dict({
    'code': [item['code'] for item in train_data],
    'label': [item['label'] for item in train_data]
})

# Initialize the tokenizer and model from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 classes: Correct, Syntax Error

# Apply tokenization to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)

# Step 4: Define Training Arguments and Train the Model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset
)

# Train the model
trainer.train()

# Step 5: Evaluation and Prediction
def predict(code):
    """Predict if the given code has a syntax error or is correct."""
    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "No error" if prediction == 0 else "Syntax error"

# Test the model with user input
print("Enter a Python code snippet (or 'exit' to quit):")
while True:
    code_input = input()
    if code_input == 'exit':
        print("Exiting the program.")
        break

    try:
        print(f"Predicted: {predict(code_input)}")
    except Exception as e:
        print(f"Error in tokenizing code: {str(e)}")
        print("Predicted: Syntax error")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
10,0.693900
20,0.675200
30,0.634700
40,0.619000
50,0.629500
60,0.543900
70,0.488300
80,0.406800
90,0.287200
100,0.208000


Step,Training Loss
10,0.693900
20,0.675200
30,0.634700
40,0.619000
50,0.629500
60,0.543900
70,0.488300
80,0.406800
90,0.287200
100,0.208000


Enter a Python code snippet (or 'exit' to quit):
print('hello')
Predicted: No error
